## 5.4 추천 상품 제안 (연관 분석)

### 공통 전처리

In [ ]:
# 공통 처리

# 불필요한 경고 메시지 무시
import warnings
warnings.filterwarnings('ignore')

# 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 한글 글꼴 설정
import platform

if platform.system() == 'Windows':
    plt.rc('font', family='Malgun Gothic')
elif platform.system() == 'Darwin':
    plt.rc('font', family='Apple Gothic')

# 데이터프레임 출력용 함수
from IPython.display import display

# 숫자 출력 조정
# 넘파이 부동소수점 출력 자리수 설정
np.set_printoptions(suppress=True, precision=4)

# 판다스 부동소수점 출력 자리수 설정
pd.options.display.float_format = '{:.4f}'.format

# 데이터프레임 모든 필드 출력
pd.set_option("display.max_columns",None)

# 그래프 글꼴 크기 설정
plt.rcParams["font.size"] = 14

# 난수 시드
random_seed = 123

데이터 집합 배포 웹 페이지  
http://archive.ics.uci.edu/ml/datasets/Online+Retail/

**데이터 필드 정보**

InvoiceNo:  주문번호  
StockCode: 상품번호  
Description: 상품설명  
Quantity: 상품개수   
InvoiceDate: 명세서_발행일  
UnitPrice:  상품단가  
CustomerID: 고객번호  
Country: 국가명  

### 5.4.4 데이터 읽어 들이기부터 데이터 확인까지

#### 데이터 읽어 들이기

In [ ]:
# 데이터 읽어 들이기
# 데이터가 엑셀 파일로 배포되므로 read_excel 함수를 사용해
# 직접 내려받아 읽어 들인다
# 시간이 꽤 걸릴 수 있으므로 주의(파일 크기 약23MB)
df = pd.read_excel('http://archive.ics.uci.edu/ml/\
machine-learning-databases/00352/Online%20Retail.xlsx')

# 우리말 필드명 정의
columns = [
    '주문번호', '상품번호', '상품설명', '상품개수', '명세서발행일', 
    '상품단가', '고객번호', '국가명'
]
df.columns = columns

#### 데이터 확인

In [ ]:
# 데이터 건수 확인
print(df.shape[0])

# 데이터 내용 확인
display(df.head())

In [ ]:
# 누락 값 확인
print(df.isnull().sum())

In [ ]:
# 국가별 건수 확인
print(df['국가명'].value_counts().head(10))

### 5.4.5 데이터 전처리
방법:   
* 주문번호의 첫 자리를 추출해 주문유형 칼럼을 추가
* 대상을 신규 주문만으로 한정
* 대상 국가를 프랑스로 한정
* 상품 간의 관계를 표로 요약

#### 대상을 신규 주문만으로 한정

In [ ]:
# '주문유형' 칼럼 추가하기

# 전처리를 위해 데이터 사본 생성
df2 = df.copy()

# 주문번호 첫 자리를 별도의 필드로 추출
# (5: 신규 주문, C: 취소 주문)
df2['주문유형'] = df2['주문번호'].map(lambda x: str(x)[0])

# 결과 확인
display(df2.head())

# 주문 유형별 건수 확인
print(df2['주문유형'].value_counts())

In [ ]:
# 신규 주문만 추출하기
df2 = df2[df2['주문유형']=='5']

# 건수 확인
print(df2.shape[0])

#### 대상 국가를 프랑스로 한정

In [ ]:
# 대상 국가를 프랑스로 한정
df3 = df2[df2['국가명']=='France']

# 건수 확인
print(df3.shape[0])

#### 주문 데이터를 멀티-핫 인코딩으로 변환

In [ ]:
# 주문번호와 상품번호를 키로 상품 개수를 집계한다
w1 = df3.groupby(['주문번호', '상품번호'])['상품개수'].sum()

# 결과 확인
print(w1.head())

In [ ]:
# 상품번호를 칼럼으로 이동(unstack 함수를 이용)
w2 = w1.unstack().reset_index().fillna(0).set_index('주문번호')

# 데이터프레임 모양 확인
print(w2.shape)

# 결과 확인
display(w2.head())

In [ ]:
# 집계 결과가 0인지 아닌지에 따라 True/False 값을 설정
basket_df = w2.apply(lambda x: x>0)

# 결과 확인
display(basket_df.head())

### 칼럼 - 상품명 딕셔너리 생성하기

다음의 코드는 [서포트 사이트](https://github.com/makaishi2/profitable_ai_book_info/blob/master/docs/C2-実習コーディング補足解説.md#C21)에서 설명을 볼 수 있습니다.  
아래 셀을 함께 실행해야 그 뒤의 셀도 정상적으로 실행할 수 있으니 주의 바랍니다.


#### 나중에 있을 분석을 위해 '상품번호'와 '상품설명'의 대응표를 만든다

In [ ]:
# '상품번호'와 '상품설명'을 추출
w3 = df2[['상품번호', '상품설명']].drop_duplicates()

# 상품번호와 상품명을 모두 문자열로 변환
w3['상품번호'] = w3['상품번호'].astype('str')
w3['상품설명'] = w3['상품설명'].astype('str')

# 상품번호를 인덱스로 삼는다
w3 = w3.set_index('상품번호')
display(w3.head())

In [ ]:
# 처음 다섯 건의 상품 코드 추출
item_list1 = w3.index[:5]

# w3에 대한 검색 결과
display(w3.loc[item_list1])

In [ ]:
# 대문자만으로 구성된 텍스트 추출

# 작업용 사본 생성
w4 = w3.copy()

# 모든 글자가 대문자인 행만 남김
w4 = w4[w4['상품설명'].map(lambda x: x.isupper())]

# 앞서 만든 item_list1로 결과를 확인
display(w4.loc[item_list1])

#### 이름이 여러 개인 상품의 처리
이름이 여러 개인 상품은 이름 중 가장 긴 것을 골라 상품명 딕셔너리에 넣는다

In [ ]:
# 가장 긴 이름을 추출

# 작업용 사본 생성
w5 = w4.copy()

# '문자 수' 칼럼 추가
w5['문자수'] = w5['상품설명'].map(len)

# 정렬
w5 = w5.sort_values(['상품번호', '문자수'], ascending=[True, False])

# 상품번호 별로 가장 긴 이름을 골라 item_dict에 추가
item_dict = w5.groupby('상품번호')['상품설명'].agg(lambda x: x[0])

# 앞서 만든 item_list1로 결과 확인
display(item_dict.loc[item_list1])

### 5.4.6 알고리즘 선택 및 분석
mlxtend 라이브러리를 이용해 아프리오리 분석 및 규칙 추출을 실습

In [ ]:
# 라이브러리 임포트
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [ ]:
# 아프리오리 분석
freq_items1 = apriori(basket_df, min_support = 0.06, 
    use_colnames = True)

# 결과 확인
display(freq_items1.sort_values('support', 
    ascending = False).head(10))

# itemset의 수를 확인
print(freq_items1.shape[0])

In [ ]:
# 연관 분석 - 규칙 추출
a_rules1 = association_rules(freq_items1, metric = "lift",
    min_threshold = 1)

# 리프트값을 기준으로 정렬
a_rules1 = a_rules1.sort_values('lift',
    ascending = False).reset_index(drop=True)

# 결과 확인
display(a_rules1.head(10))

# 추출된 규칙 수 확인
print(a_rules1.shape[0])

In [ ]:
# 관계성이 높은 상품의 상품번호 리스트
item_list = ['23254', '23256', '22726', '22727', '22728']

# 상품명 확인
for item in item_list:
    print(item, item_dict[item])

### 5.4.7 튜닝
min_support 값을 0.065로 바꾸고 같은 분석을 반복한다

In [ ]:
# 아프리오리 분석
freq_items2 = apriori(basket_df, min_support = 0.065,
    use_colnames = True)

# 규칙 추출
a_rules2 = association_rules(freq_items2, metric = "lift",
    min_threshold = 1)

# 리프트값을 기준으로 내림차순 정렬
a_rules2 = a_rules2.sort_values('lift',
    ascending = False).reset_index(drop=True)

# 결과 확인
display(a_rules2.head(10))

두 개의 아프리오리 분석 결과 중 '23254', '23256'을 엮는 항목을 추출한다

In [ ]:
# 조사 대상 정의
t_set = set([23254, 23256])

# 첫 번째 분석 결과에서 대상 항목 추출
idx1 = freq_items1['itemsets'].map(
    lambda x: not x.isdisjoint(t_set))
item1 = freq_items1[idx1]

# 두 번째 분석 결과에서 대상 항목 추출
idx2 = freq_items2['itemsets'].map(
    lambda x: not x.isdisjoint(t_set))
item2 = freq_items2[idx2]

# 결과 확인
display(item1)
display(item2)

### 5.4.8 관계 그래프 시각화하기
상품 간의 관계를 관계 그래프로 시각화한다.
설명은 [서포트 사이트](https://github.com/makaishi2/profitable_ai_book_info/blob/master/docs/C2-実習コーディング補足解説.md#C22)를 참조하라.

#### 노드 추출하기

In [ ]:
# 리프트값 상위 20개를 대상으로 삼는다
a = a_rules1.head(20)

# 부모 노드 추출
ant = a['antecedents'].values
ant = [tuple(x) for x in ant]

# 자식 노드 추출
con = a['consequents'].values
con = [tuple(x) for x in con]

# 모든 노드를 열거
both = ant + con
both = list(set(both))

# 결과의 일부를 확인
print(both[:10])

#### 관계 그래프 구성하기

In [ ]:
# networkx 라이브러리 임포트
import networkx as nx

# 관계 그래프 초기화
G = nx.DiGraph()

# 노드 추가
for n in both:
  G.add_node(n)

# 엣지 추가
for i in range(len(a)):
    item = a.loc[i]
    ant = tuple(item['antecedents'])
    con = tuple(item['consequents'])
    G.add_edge(ant, con)

#### 그래프 그리기

In [ ]:
# 그래프 그리기
pos = nx.spring_layout(G, k=0.6, seed=random_seed)

plt.figure(figsize=(8, 8))
nx.draw_networkx_nodes(G, pos)
nx.draw_networkx_edges(G, pos)
nx.draw_networkx_labels(G, pos,
    horizontalalignment='left', 
    verticalalignment='center')
plt.axis('off')
plt.tight_layout()
plt.show()